### Enriquecimiento de datos con Google Places API

Este código permite trabajar con un CSV limpio, generar una muestra representativa por código postal 
y enriquecerla con datos de competencia y puntuación media de locales similares usando la API de Google Places.

In [6]:
import pandas as pd
import os
import googlemaps
from dotenv import load_dotenv
import time

# Cargar variables de entorno
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
gmaps = googlemaps.Client(key=api_key)

# Cargar CSV limpio y crear muestra
print("Cargando CSV limpio...")
df = pd.read_csv('../data/locales_procesado.csv')
muestra = df.groupby('codigo_postal', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42))
muestra.reset_index(drop=True, inplace=True)
muestra.to_csv('../data/locales_procesado_muestra.csv', index=False)

print("Archivo 'data/locales_procesado_muestra.csv' creado con éxito.")

Cargando CSV limpio...
Archivo 'data/locales_procesado_muestra.csv' creado con éxito.


/var/folders/py/btwd7pk93f5g87hjsfgb3wmw0000gn/T/ipykernel_58586/2648155688.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  muestra = df.groupby('codigo_postal', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42))


In [8]:
# Consultar la API de Google Places para cada fila
muestra['competencia_count'] = 0
muestra['competencia_rating'] = 0.0

for index, row in muestra.iterrows():
    try:
        latitude = row['latitud']
        longitude = row['longitud']
        categoria_negocio = row['tipo_negocio']
        
        places_result = gmaps.places_nearby(
            location=(latitude, longitude),
            radius=500,
            type=categoria_negocio
        )
        
        count = len(places_result['results'])
        ratings = [place.get('rating') for place in places_result['results'] if 'rating' in place]
        avg_rating = sum(ratings) / len(ratings) if ratings else 0.0
        
        muestra.at[index, 'competencia_count'] = count
        muestra.at[index, 'competencia_rating'] = avg_rating
        
        time.sleep(1)
        
    except Exception as e:
        print(f"Error en la fila {index}: {e}")

# Guardar archivo final enriquecido
muestra.to_csv('../data/locales_enriquecido_muestra.csv', index=False)
print("Archivo 'data/locales_enriquecido_muestra.csv' creado con éxito.")

Archivo 'locales_enriquecido_muestra.csv' creado con éxito.


In [10]:
# Cargar el CSV enriquecido
df = pd.read_csv('../data/locales_enriquecido_muestra.csv')

# Revisar las primeras filas
print(df.head())

# Revisar las columnas
df.info()

# Revisar si hay valores nulos
print(df.isnull().sum())

# Revisar estadísticas básicas
print(df.describe())

# Verificar duplicados
duplicados = df.duplicated()
print(f"Duplicados encontrados: {duplicados.sum()}")

                             nombre                                direccion  \
0  Hotel NH Collection Madrid Colón  Calle del Marqués de Zurgena, 4, Madrid   
1                 Hotel Vincci Soma                Calle de Goya, 79, Madrid   
2              El Corte Inglés Goya          Calle de Goya, Sótano 1, Madrid   
3                AC Hotel Recoletos           Calle de Recoletos, 18, Madrid   
4           Only YOU Boutique Hotel          Calle del Barquillo, 21, Madrid   

   codigo_postal  puntuacion_media  numero_reviews  \
0          28001               4.6          1504.0   
1          28001               4.3          2867.0   
2          28001               4.2         25800.0   
3          28001               4.5           587.0   
4          28001               4.6          2731.0   

                                      tipo_negocio    latitud  longitud  \
0        lodging, point_of_interest, establishment  40.426076 -3.689070   
1        lodging, point_of_interest, establi